In [1]:
from transformers import AutoModelForSequenceClassification, pipeline, AutoTokenizer
import shap

2022-09-09 00:13:05,679 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(

2022-09-09 00:13:05,684 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(

2022-09-09 00:13:05,686 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\tensor_shap

2022-09-09 00:13:05,765 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\types_pb2.py:81: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueDescriptor(

2022-09-09 00:13:05,769 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\types_pb2.py:85: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueDescriptor(

2022-09-09 00:13:05,773 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\types_pb2.py:89: Dep

2022-09-09 00:13:05,848 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\types_pb2.py:161: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueDescriptor(

2022-09-09 00:13:05,851 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\types_pb2.py:165: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueDescriptor(

2022-09-09 00:13:05,855 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\types_pb2.py:169: 

2022-09-09 00:13:05,934 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\resource_handle_pb2.py:76: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(

2022-09-09 00:13:05,938 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\resource_handle_pb2.py:83: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(

2022-09-09 00:13:05,942 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\resource_handle_

2022-09-09 00:13:06,021 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\tensor_pb2.py:124: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(

2022-09-09 00:13:06,026 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\tensor_pb2.py:131: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(

2022-09-09 00:13:06,030 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\tensorflow\core\framework\tensor_pb2.py:138: DeprecationWa

2022-09-09 00:13:08,875 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

2022-09-09 00:13:09,583 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp

2022-09-09 00:13:10,495 - py.warnings - WARNING - C:\Users\frede\anaconda3\envs\unhcr-hate-speech-kedro\lib\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,

2022-09-09 00:13:10,496 - py.warnings - WARNING - C:\Users

## Load Dataset

In [5]:
unhcr_data = catalog.load("test_unhcr")

2022-09-09 00:17:32,014 - kedro.io.data_catalog - INFO - Loading data from 'test_unhcr' (ParquetDataSet)...


In [6]:
unhcr_data.head()

,text,label,dataset
0,afganlaries we are not able to see suriyeleris...,2,unhcr
1,afghanistan is to biden as syria is to obama. ...,2,unhcr
2,refugeeswelcome is a terrorist organisation th...,2,unhcr
3,@ abcd76711747 boys owniest eimmigrant immigra...,2,unhcr
4,"@USER @USER most are not refugees, they are bo...",2,unhcr


## Create Transformer Pipeline

Download tokenizer

In [11]:
def _download_tokenizer(tokenizer_identifier: str = "unhcr/hatespeech-detection"):
    """
    Downloads the tokenizer that is associated
    with the model from https://huggingface.co/unhcr/hatespeech-detection

    Parameters
    ----------
    tokenizer_identifier : str
        The name of Huggingface project that the tokenizer is associated with

    Returns
    -------
    tokenizer
        Huggingface transformer tokenizer
    """
    return AutoTokenizer.from_pretrained(tokenizer_identifier)

Download model

In [12]:
def _download_classifier(classifier_identifier: str = "unhcr/hatespeech-detection"):
    """
    Downloads the model from https://huggingface.co/unhcr/hatespeech-detection

    Parameters
    ----------
    classifier_identifier : str
        The name of Huggingface project that the model is associated with

    Returns
    -------
    model
        Huggingface transformer model

    Dict
        Label mapping from ids to labels

    """
    id2label = {
        0: "Normal",
        1: "Offensive",
        2: "Hate speech",
    }

    label2id = {id2label[i]: i for i in id2label}

    # TODO: add num_labels to config
    model = AutoModelForSequenceClassification.from_pretrained(
        classifier_identifier, num_labels=3, id2label=id2label, label2id=label2id
    )

    return model, id2label

Create pipeline

In [15]:
local = True
tokenizer = _download_tokenizer()
classifier, id2label = _download_classifier()

inference_pipeline = pipeline(
    task="text-classification",
    tokenizer=tokenizer,
    model=classifier,
    device=-1 if local else 0,
    top_k=3,
    max_length=128,
    padding=True,
    truncation=True,
)

## SHAP Analysis

The code below takes long time to run. It is advised that this is run on a GPU or large CPU.

For the purpose of the example, shap values are only calculated for two text samples.

In [21]:
explainer = shap.Explainer(inference_pipeline, inference_pipeline.tokenizer)
shap_values = explainer(unhcr_data["text"][:2])

2022-09-09 00:31:20,563 - py.warnings - WARNING - `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

2022-09-09 00:31:20,564 - py.warnings - WARNING - `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

2022-09-09 00:31:20,568 - py.warnings - WARNING - `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is saf

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer:  50%|███████████████████████████████████████████████████▌                                                   | 1/2 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 3it [03:03, 91.62s/it]                                                                                                            

2022-09-09 00:34:23,795 - py.warnings - WARNING - `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

2022-09-09 00:34:23,799 - py.warnings - WARNING - `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



### Sentence-level: Text

In [ ]:
# Plot the results for the first sentence
shap.plots.text(shap_values[0])

### Sentence-level: Waterfall

In [ ]:
shap.plots.waterfall(shap_values[0])

### Top Most Important Features for the Hate Speech Class

In [ ]:
shap.plots.bar(shap_values[:,:,2].mean(0), order=shap.Explanation.argsort.flip, max_display=15)